In [22]:
import pandas as pd 
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt 

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from tqdm import tqdm

In [23]:
class Model1 (nn.Module):
    def __init__(self):
        super(Model1,self).__init__()
        self.model = nn.Sequential(
            nn.Linear(5,3),
            nn.LeakyReLU(negative_slope=0.1),
            nn.Linear(3,1),
            nn.Tanh()
        )
    def forward(self,x):
        return self.model(x)

In [24]:
df = pd.read_json("data2/normal_vpm.json",orient="records",lines=True)


In [25]:
list_data = df.to_numpy()
np.random.shuffle(list_data)

train_x = torch.Tensor(list_data[:10000,:-1])
train_Y = torch.Tensor(list_data[:10000,-1].reshape(-1,1))
train_data = TensorDataset(train_x,train_Y)
train_dataLoader = DataLoader(train_data,batch_size=128,shuffle=True)

test_x = torch.Tensor(list_data[10000:,:-1])
test_Y = torch.Tensor(list_data[10000:,-1].reshape(-1,1))
test_dataset = TensorDataset(test_x,test_Y)
test_dataLoader = DataLoader(test_dataset,batch_size=128,shuffle=True)


In [26]:
test_x[0:10]

tensor([[2.8666e-05, 2.0000e+00, 5.3097e-01, 2.0000e+01, 2.0000e+01],
        [4.2209e-01, 5.0000e+00, 9.2192e-05, 1.0000e+01, 0.0000e+00],
        [3.7185e-01, 3.0000e+00, 6.2788e-05, 2.0000e+01, 2.0000e+01],
        [7.1701e-01, 4.0000e+00, 4.3417e-05, 5.0000e+00, 0.0000e+00],
        [5.4183e-01, 7.0000e+00, 1.0055e-04, 1.5000e+01, 0.0000e+00],
        [9.8292e-01, 8.0000e+00, 6.3343e-05, 1.5000e+01, 0.0000e+00],
        [5.2095e-01, 5.0000e+00, 7.4696e-05, 2.0000e+01, 2.0000e+01],
        [1.3578e-01, 2.0000e+00, 1.1463e-04, 5.0000e+00, 0.0000e+00],
        [2.6731e-01, 4.0000e+00, 1.1646e-04, 1.5000e+01, 1.5000e+01],
        [2.0396e-01, 4.0000e+00, 1.5263e-04, 1.0000e+01, 1.0000e+01]])

In [27]:
model1 = Model1()
optimizer = optim.Adam(model1.parameters(),lr=0.06)
criterion = nn.MSELoss()

In [28]:
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")
model1.to(device)
print("Running model on:",device)

Running model on: cpu


In [29]:
epochs = 20

model1.train()
for epoch in tqdm(range(epochs)):
    train_loss = []
    for x, Y in train_dataLoader:
        x = x.to(device)
        Y = Y.to(device)
        optimizer.zero_grad()
        out = model1(x)
        loss = criterion(out, Y)
        loss.backward()
        optimizer.step()
        train_loss.append(loss.item())
    print(f"Loss: {np.average(train_loss):.3f}")

 10%|█         | 2/20 [00:00<00:01, 11.98it/s]Loss: 0.354
Loss: 0.304
Loss: 0.255
 20%|██        | 4/20 [00:00<00:01, 12.12it/s]Loss: 0.276
Loss: 0.258
 40%|████      | 8/20 [00:00<00:00, 13.40it/s]Loss: 0.241
Loss: 0.248
Loss: 0.237
 50%|█████     | 10/20 [00:00<00:00, 13.54it/s]Loss: 0.244
Loss: 0.231
 70%|███████   | 14/20 [00:01<00:00, 13.63it/s]Loss: 0.231
Loss: 0.234
Loss: 0.238
 80%|████████  | 16/20 [00:01<00:00, 13.80it/s]Loss: 0.227
Loss: 0.231
100%|██████████| 20/20 [00:01<00:00, 13.51it/s]Loss: 0.228
Loss: 0.242



In [30]:
correct = 0
for x, Y in test_dataLoader:
    model1.eval()
    x = x.to(device)
    Y = Y.cpu()
    out=model1(x)
    out.cpu()
    for i,o in enumerate(out):
        if o * Y[i] > 0:
            correct += 1
print(f'Corectly predicted: {correct} out of {len(test_dataset)}\nTest accuracy: {correct/len(test_dataset):.2%}')


Corectly predicted: 2562 out of 2808
Test accuracy: 91.24%


In [32]:
model1.cpu()
print(model1(torch.Tensor(np.array([[0.005,4,0.00003,0,0],[0.005,4,0.00003,0,5],[0.005,4,0.00003,0,10],[0.005,4,0.00003,0,15],[0.005,4,0.00003,0,20]]))))
#print(model1(torch.Tensor(np.array([60000000,2,0,0]))))

tensor([[0.1641],
        [0.9871],
        [0.9977],
        [0.9996],
        [0.9999]], grad_fn=<TanhBackward>)
